In [1]:
import requests
import pandas as pd
import polars as pl
import duckdb

from hashlib import md5
from pathlib import Path

In [2]:
url_usr = "https://raw.githubusercontent.com/danielmiessler/SecLists/refs/heads/master/Usernames/Names/names.txt"
url_pwd = "https://raw.githubusercontent.com/danielmiessler/SecLists/refs/heads/master/Passwords/500-worst-passwords.txt"

In [3]:
resp_usr = requests.get(url_usr)
resp_pwd = requests.get(url_pwd)

In [4]:
for resp in (resp_usr, resp_pwd):
    assert resp.status_code == 200

In [5]:
with open("data.csv", mode="w", encoding="utf-8") as f:
    f.write("usr,pwd\n")  # Write the header row
    for usr in filter(None, map(lambda b: b.decode("utf-8").strip(), resp_usr.content.split(b"\n"))):
        for pwd in filter(None, map(lambda b: b.decode("utf-8").strip(), resp_pwd.content.split(b"\n"))):
            f.write(f"{usr},{pwd}\n")


In [6]:
path = Path().cwd() / "data.csv"
assert path.exists()

In [7]:
%%time
pl_dataframe = pl.read_csv(path)#.drop_nans()
pl_dataframe.schema

CPU times: user 299 ms, sys: 38.3 ms, total: 337 ms
Wall time: 105 ms


Schema([('usr', String), ('pwd', String)])

In [8]:
%%time
pl_lazyframe = pl.scan_csv(path)#.drop_nans()

CPU times: user 475 μs, sys: 1.16 ms, total: 1.63 ms
Wall time: 3.45 ms


In [9]:
%%time
pd_dataframe = pd.read_csv(path).dropna()

CPU times: user 666 ms, sys: 65.4 ms, total: 731 ms
Wall time: 745 ms


In [10]:
def md5_hash(value: str) -> str:
    return md5(value.encode()).hexdigest()

In [11]:
%%time

for col in ["usr", "pwd"]:
    pd_dataframe[f"{col}_md5"] = pd_dataframe[col].apply(md5_hash)


CPU times: user 5.28 s, sys: 155 ms, total: 5.43 s
Wall time: 5.46 s


In [12]:

pl_dataframe = pl_dataframe.with_columns(
     pl.col("usr").map(md5_hash).alias("usr_md5")
)

AttributeError: 'Expr' object has no attribute 'map'

In [ ]:
pl_dataframe

usr,pwd,usr_md5
str,str,str
"""aaliyah""","""123456""","""aaliyah"""
"""aaliyah""","""password""","""aaliyah"""
"""aaliyah""","""12345678""","""aaliyah"""
"""aaliyah""","""1234""","""aaliyah"""
"""aaliyah""","""pussy""","""aaliyah"""
…,…,…
"""zylen""","""mistress""","""zylen"""
"""zylen""","""phantom""","""zylen"""
"""zylen""","""billy""","""zylen"""
